<a href="https://colab.research.google.com/github/manojjaiss/Dsinternship/blob/main/Computer_Vision_%26_Food_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set Up Your Google Colab Environment**

In [1]:
!pip install tensorflow matplotlib scikit-learn pandas


**Download and Extract the Food-101 Dataset**

In [2]:
import os
import tarfile
import urllib.request

# Create a directory to store data
data_dir = "food-101"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the dataset
url = "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"
filename = os.path.join(data_dir, "food-101.tar.gz")

if not os.path.exists(filename):
    print("Downloading dataset...")
    urllib.request.urlretrieve(url, filename)

# Extract dataset
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall(path=data_dir)
    print("Extraction complete.")


Extraction complete.


**Prepare the Dataset for CNN**


Load data paths and split

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = "food-101/food-101/images"

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


** Build and Train CNN Model**

 CNN using TensorFlow

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    14,745,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        25,957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,791,205 (56.42 MB)

 Trainable params: 14,791,205 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Model**



In [5]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2908s 1s/step - accuracy: 0.0231 - loss: 4.5699 - val_accuracy: 0.0837 - val_loss: 4.1357
Epoch 2/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2952s 1s/step - accuracy: 0.0818 - loss: 4.1191 - val_accuracy: 0.1272 - val_loss: 3.8469
Epoch 3/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2897s 1s/step - accuracy: 0.1258 - loss: 3.8043 - val_accuracy: 0.1388 - val_loss: 3.7787
Epoch 4/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2866s 1s/step - accuracy: 0.1716 - loss: 3.5188 - val_accuracy: 0.1573 - val_loss: 3.6726
Epoch 5/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2889s 1s/step - accuracy: 0.2265 - loss: 3.1934 - val_accuracy: 0.1556 - val_loss: 3.7051
Epoch 6/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2903s 1s/step - accuracy: 0.2839 - loss: 2.8644 - val_accuracy: 0.1472 - val_loss: 3.8068
Epoch 7/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2875s 1s/step - accuracy: 0.3472 - loss: 2.5397 - val_accuracy: 0.1446 - val_loss: 3.9461
Epoch 8/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2904s 1s/step - accuracy: 0.4079 -

**Predict Food Class**

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_food(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    class_index = np.argmax(pred)
    class_label = list(train_generator.class_indices.keys())[class_index]
    return class_label

# Example
predict_food('food-101/food-101/images/sushi/1238583.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step


'deviled_eggs'

**Predict Salt Content Based on Food Class**

Load the Salt Content Dataset (food_raw.csv)

In [16]:
import pandas as pd

# Replace with your actual path if needed
df = pd.read_csv('/content/food_raw.csv')

# View the column names
print(df.columns)


Index(['No.', 'Food_ID', 'Nama Bahan Makanan', 'Fast_Food', 'Sumber', 'Tipe',
       'Jenis_Olahan', 'Air (g)', 'Energi (Kal)', 'Protein (g)', 'Lemak (g)',
       'Karbohidrat (g)', 'Serat (g)', 'Abu (g)', 'Kalsium (Ca) (mg)',
       'Fosfor (P) (mg)', 'Besi (Fe) (mg)', 'Natrium (Na) (mg)',
       'Kalium (Ka) (mg)', 'Tembaga (Cu) (mg)', 'Seng (Zn) (mg)',
       'Retinol (vit. A) (mcg)', 'β-karoten (mcg)', 'Karoten total (mcg)',
       'Thiamin (vit. B1) (mg)', 'Riboflavin (vit. B2) (mg)', 'Niasin (mg)',
       'Vitamin C (mg)', 'BDD (%)', 'Mentah / Olahan', 'Kelompok Makanan',
       'Sumber TKPI 2019', 'Fast_Food_New', 'Tipe_New', 'Jenis_Olahan_New',
       'Mentah / Olahan_New', 'Kelompok Makanan_New', 'Fast_Food_New_0',
       'Fast_Food_New_1', 'Tipe_New_0', 'Tipe_New_1', 'Jenis_Olahan_New_0',
       'Jenis_Olahan_New_1', 'Jenis_Olahan_New_2', 'Jenis_Olahan_New_3',
       'Jenis_Olahan_New_4', 'Jenis_Olahan_New_5', 'Jenis_Olahan_New_6',
       'Jenis_Olahan_New_7', 'Jenis_Olahan_N

Create a Lookup Function for Salt Content

In [17]:
def get_salt_content(food_label):
    # Match predicted food name with entries in the dataset
    matches = df[df['food_name'].str.lower().str.contains(food_label.lower())]

    if not matches.empty:
        avg_salt = matches['sodium'].mean()
        return round(avg_salt, 2)
    else:
        return "Not available"


Adjust Electrical Stimulation Based on Salt

In [20]:
import pandas as pd

def get_salt_content(food_label):
    """
    Gets the average salt content for a given food label from the dataframe.

    Args:
        food_label (str): The name of the food.

    Returns:
        float: The average salt content in grams, or "Not available" if not found.
    """
    # Assuming 'Nama Bahan Makanan' is the correct column for food names
    matches = df[df['Nama Bahan Makanan'].str.lower().str.contains(food_label.lower())]

    if not matches.empty:
        # Assuming 'Sodium (mg)' is the correct column for sodium content
        avg_salt = matches['Sodium (mg)'].mean()
        # Convert milligrams to grams
        return round(avg_salt / 1000, 2)
    else:
        return "Not available"

 Electrical stimulation Example

In [22]:
import pandas as pd

def get_salt_content(food_label):
    """
    Gets the average salt content for a given food label from the dataframe.

    Args:
        food_label (str): The name of the food.

    Returns:
        float: The average salt content in grams, or "Not available" if not found.
    """
    # Assuming 'Nama Bahan Makanan' is the correct column for food names
    matches = df[df['Nama Bahan Makanan'].str.lower().str.contains(food_label.lower())]

    if not matches.empty:
        # Assuming 'Sodium (mg)' is the correct column for sodium content
        avg_salt = matches['Sodium (mg)'].mean()
        # Convert milligrams to grams
        return round(avg_salt / 1000, 2)
    else:
        return "Not available"

def adjust_stimulation(salt_content):
    if salt_content == "Not available":
        return "Default stimulation mode"
    elif salt_content > 2.0:
        return "Reduce stimulation intensity"
    elif salt_content < 0.5:
        return "Increase stimulation intensity"
    else:
        return "Normal stimulation"

# Predict the food from an example image
predicted_food = predict_food('food-101/food-101/images/sushi/1238583.jpg')

# Get the salt estimate for the predicted food
salt_estimate = get_salt_content(predicted_food) # Assigning the result to salt_estimate

# Example
stim_msg = adjust_stimulation(salt_estimate)
print(f"Electrical stimulation: {stim_msg}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Electrical stimulation: Default stimulation mode


Final Combined Pipeline Example

In [24]:
import pandas as pd

# Assuming your CSV file is named 'food_raw.csv' and located in the current directory
df = pd.read_csv('/content/food_raw.csv')

# View the column names to verify if 'Sodium (mg)' exists
print(df.columns)

# Get a random food item from the dataset
food_item = df.sample(1)

# Extract the food name and salt content using the correct column name
food_name = food_item['Nama Bahan Makanan'].values[0]

# Check if 'Sodium (mg)' exists in the columns before accessing it
if 'Sodium (mg)' in food_item.columns:
    salt_content_mg = food_item['Sodium (mg)'].values[0]
else:
    # Handle the case where the column is not found, e.g., print an error message or assign a default value
    print("Error: 'Sodium (mg)' column not found in the dataframe.")
    salt_content_mg = 0  # Assign a default value or handle the error appropriately


# Convert salt content to grams
salt_content_g = salt_content_mg / 1000

# Print the information
print(f"Food: {food_name}")
print(f"Salt content (mg): {salt_content_mg}")
print(f"Salt content (g): {salt_content_g:.2f}")

Index(['No.', 'Food_ID', 'Nama Bahan Makanan', 'Fast_Food', 'Sumber', 'Tipe',
       'Jenis_Olahan', 'Air (g)', 'Energi (Kal)', 'Protein (g)', 'Lemak (g)',
       'Karbohidrat (g)', 'Serat (g)', 'Abu (g)', 'Kalsium (Ca) (mg)',
       'Fosfor (P) (mg)', 'Besi (Fe) (mg)', 'Natrium (Na) (mg)',
       'Kalium (Ka) (mg)', 'Tembaga (Cu) (mg)', 'Seng (Zn) (mg)',
       'Retinol (vit. A) (mcg)', 'β-karoten (mcg)', 'Karoten total (mcg)',
       'Thiamin (vit. B1) (mg)', 'Riboflavin (vit. B2) (mg)', 'Niasin (mg)',
       'Vitamin C (mg)', 'BDD (%)', 'Mentah / Olahan', 'Kelompok Makanan',
       'Sumber TKPI 2019', 'Fast_Food_New', 'Tipe_New', 'Jenis_Olahan_New',
       'Mentah / Olahan_New', 'Kelompok Makanan_New', 'Fast_Food_New_0',
       'Fast_Food_New_1', 'Tipe_New_0', 'Tipe_New_1', 'Jenis_Olahan_New_0',
       'Jenis_Olahan_New_1', 'Jenis_Olahan_New_2', 'Jenis_Olahan_New_3',
       'Jenis_Olahan_New_4', 'Jenis_Olahan_New_5', 'Jenis_Olahan_New_6',
       'Jenis_Olahan_New_7', 'Jenis_Olahan_N

***Automated Dietary Recommendation System***

*Dietary Rules & Recommendations Logic*

In [25]:
def dietary_recommendation(food_label):
    matches = df[df['food_name'].str.lower().str.contains(food_label.lower())]

    if matches.empty:
        return "⚠️ No data found. Please consume in moderation."

    # Take averages in case multiple rows match
    avg_salt = matches['sodium'].mean()
    avg_calories = matches['calories'].mean()
    avg_fat = matches['fat'].mean()

    advice = f"🍽️ Nutrition for {food_label.title()}:\n"
    advice += f"   - Salt: {avg_salt:.2f}g\n"
    advice += f"   - Calories: {avg_calories:.0f} kcal\n"
    advice += f"   - Fat: {avg_fat:.2f}g\n\n"

    # Simple rules
    if avg_salt > 2.0 or avg_fat > 20 or avg_calories > 500:
        advice += "⚠️ High in salt/fat/calories. Recommend limited intake.\n"
    else:
        advice += "✅ Balanced choice. Safe for regular consumption.\n"

    # Suggest healthy substitutes (example-based)
    substitutions = {
        "burger": "grilled chicken sandwich",
        "pizza": "vegetable flatbread",
        "fried": "steamed or grilled dishes",
        "samosa": "baked veggie roll"
    }

    for word, sub in substitutions.items():
        if word in food_label.lower():
            advice += f"💡 Try a healthier option like: *{sub.title()}*\n"

    return advice


***Full Integrated Pipeline***

In [26]:
def full_diet_pipeline(image_path):
    print("🔍 Identifying food...")
    label = predict_food(image_path)
    print(f"🍕 Predicted food: {label}\n")

    salt = get_salt_content(label)
    stim = adjust_stimulation(salt)
    print(f"🧂 Salt content: {salt}g")
    print(f"⚡ Electrical Adjustment: {stim}\n")

    # Nutrition + dietary advice
    print(dietary_recommendation(label))


*pipeline_arjustment*

In [29]:
import pandas as pd

def dietary_recommendation(food_label):
    """
    Provides dietary recommendations based on the food label.

    Args:
        food_label (str): The name of the food.

    Returns:
        str: Dietary advice for the food.
    """
    # Access the correct column name for food names, assuming it's 'Nama Bahan Makanan'
    matches = df[df['Nama Bahan Makanan'].str.lower().str.contains(food_label.lower())]

    if matches.empty:
        return "⚠️ No data found. Please consume in moderation."

    # Assuming correct column names for sodium, calories, and fat
    avg_salt = matches['Sodium (mg)'].mean() / 1000  # Convert to grams
    avg_calories = matches['Energi (kal)'].mean()
    avg_fat = matches['Lemak (g)'].mean()

    advice = f"🍽️ Nutrition for {food_label.title()}:\n"
    advice += f"   - Salt: {avg_salt:.2f}g\n"
    advice += f"   - Calories: {avg_calories:.0f} kcal\n"
    advice += f"   - Fat: {avg_fat:.2f}g\n\n"

    # Simple rules
    if avg_salt > 2.0 or avg_fat > 20 or avg_calories > 500:
        advice += "⚠️ High in salt/fat/calories. Recommend limited intake.\n"
    else:
        advice += "✅ Balanced choice. Safe for regular consumption.\n"

    # Suggest healthy substitutes (example-based)
    substitutions = {
        "burger": "grilled chicken sandwich",
        "pizza": "vegetable flatbread",
        "fried": "steamed or grilled dishes",
        "samosa": "baked veggie roll"
    }

    for word, sub in substitutions.items():
        if word in food_label.lower():
            advice += f"💡 Try a healthier option like: *{sub.title()}*\n"

    return advice

*full_diet_pipeline*

In [33]:
import pandas as pd
from tensorflow.keras.preprocessing import image
import numpy as np


def full_diet_pipeline(image_path, model, train_generator, df):
    """
    Performs a full dietary analysis pipeline on an image of food.

    Args:
        image_path (str): The path to the image file.
        model: The trained Keras model for food prediction.
        train_generator: The Keras ImageDataGenerator used for training.
        df: The Pandas DataFrame containing food nutritional information.

    Returns:
        None: Prints the predicted food, salt content, electrical stimulation adjustment,
              and dietary recommendations.
    """
    print("🔍 Identifying food...")
    label = predict_food(image_path, model, train_generator)  # Pass necessary arguments
    print(f"🍕 Predicted food: {label}\n")

    salt = get_salt_content(label, df)  # Pass df as argument
    stim = adjust_stimulation(salt)
    print(f"🧂 Salt content: {salt}g")
    print(f"⚡ Electrical Adjustment: {stim}\n")

    # Nutrition + dietary advice
    print(dietary_recommendation(label, df))  # Pass df as argument


def predict_food(img_path, model, train_generator):
    """
    Predicts the food in an image using a pre-trained model.

    Args:
        img_path (str): Path to the image.
        model: The trained Keras model.
        train_generator: The Keras ImageDataGenerator used for training.

    Returns:
        str: The predicted food label.
    """
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    class_index = np.argmax(pred)
    class_label = list(train_generator.class_indices.keys())[class_index]
    return class_label


def get_salt_content(food_label, df):
    """
    Gets the average salt content for a given food label from the dataframe.

    Args:
        food_label (str): The name of the food.
        df: The Pandas DataFrame containing nutritional information.

    Returns:
        float: The average salt content in grams, or "Not available" if not found.
    """
    matches = df[df['Nama Bahan Makanan'].str.lower().str.contains(food_label.lower())]

    if not matches.empty:
        avg_salt = matches['Sodium (mg)'].mean()
        return round(avg_salt / 1000, 2)  # Convert to grams
    else:
        return "Not available"


def adjust_stimulation(salt_content):
    """
    Adjusts the electrical stimulation intensity based on salt content.

    Args:
        salt_content (float): The salt content of the food in grams.

    Returns:
        str: A message indicating the stimulation adjustment.
    """
    if salt_content == "Not available":
        return "Default stimulation mode"
    elif salt_content > 2.0:
        return "Reduce stimulation intensity"
    elif salt_content < 0.5:
        return "Increase stimulation intensity"
    else:
        return "Normal stimulation"


def dietary_recommendation(food_label, df):
    """
    Provides dietary recommendations based on the food label.

    Args:
        food_label (str): The name of the food.
        df: The Pandas DataFrame containing nutritional information.

    Returns:
        str: Dietary advice for the food.
    """
    matches = df[df['Nama Bahan Makanan'].str.lower().str.contains(food_label.lower())]

    if matches.empty:
        return "⚠️ No data found. Please consume in moderation."

    avg_salt = matches['Sodium (mg)'].mean() / 1000  # Convert to grams
    avg_calories = matches['Energi (kal)'].mean()
    avg_fat = matches['Lemak (g)'].mean()

    advice = f"🍽️ Nutrition for {food_label.title()}:\n"
    advice += f"   - Salt: {avg_salt:.2f}g\n"
    advice += f"   - Calories: {avg_calories:.0f} kcal\n"
    advice += f"   - Fat: {avg_fat:.2f}g\n\n"

    if avg_salt > 2.0 or avg_fat > 20 or avg_calories > 500:
        advice += "⚠️ High in salt/fat/calories. Recommend limited intake.\n"
    else:
        advice += "✅ Balanced choice. Safe for regular consumption.\n"

    substitutions = {
        "burger": "grilled chicken sandwich",
        "pizza": "vegetable flatbread",
        "fried": "steamed or grilled dishes",
        "samosa": "baked veggie roll"
    }

    for word, sub in substitutions.items():
        if word in food_label.lower():
            advice += f"💡 Try a healthier option like: *{sub.title()}*\n"

    return advice

*example*

In [34]:
# Assuming you have loaded your model, train_generator, and df
full_diet_pipeline('food-101/food-101/images/french_fries/1020588.jpg', model, train_generator, df)

🔍 Identifying food...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
🍕 Predicted food: french_fries

🧂 Salt content: Not availableg
⚡ Electrical Adjustment: Default stimulation mode

⚠️ No data found. Please consume in moderation.
